In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import plot_roc_curve
from datetime import datetime
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from joblib import dump

In [ ]:
#Read in the data
df = pd.read_csv("../Logged-Matrix-Data.csv")
X = df[['Bluetooth-2','Accelerometer-1', 'Accelerometer-2', 'Accelerometer-3', 'Gyroscope-1', 'Gyroscope-2', 'Gyroscope-3']]
df[['Distance']] = df[['Distance']].values > 6
y = df[['Distance']]

#Split the training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y)
#Series where metrics are stored
s = pd.Series(index = ["id", "validation score", "auc", "f-measure", "precision", "recall"])

In [ ]:
#Call the model
model = GaussianNB()
#Train the model
model.fit(X_train, y_train)
#Test the model
y_hats = model.predict(X_test)
plot_roc_curve(model,X_test, y_test['Distance'])
#Collect and print model metrics
s["validation score"] = roc_auc_score(y_test['Distance'], y_hats)
print(s["validation score"])

In [ ]:
#Set the IDs for the model
model_name = "Naive_Bayes"
dateId = datetime.now().strftime('%m_%d_%Y_%H_%M_%S')
#Read in the pi data for validation
df_test = pd.read_csv("../Pi_Distance_Data.csv")
X_pi = df_test[['RSSI','ax','ay','az','gx','gy','gz']]
df_test[['Distance']] = df_test[['Distance']].values > 6
y_pi = df_test[['Distance']]
y_hats = model.predict(X_pi)
#Collect and print the model metrics
s["auc"] = roc_auc_score(y_pi['Distance'], y_hats)
print(s["auc"])
plot_roc_curve(model,X_pi, y_pi['Distance'])
s["id"] = dateId
s["f-measure"] = f1_score(y_pi['Distance'], y_hats)
print(s["f-measure"])
s["precision"] = precision_score(y_pi['Distance'], y_hats)
print(s["precision"])
s["recall"] = recall_score(y_pi['Distance'], y_hats)
print(s["recall"])
#Append this data to the CSV
pd.DataFrame(s).T.to_csv(model_name+".csv", mode ="a", index=False, header = False)
#Pickle the model
dump(model, "saved/" + model_name+'/' + dateId + "_" + str(s["auc"]) + '.joblib')